In [86]:
import data_engine as de
import pandas as pd
from feature_engine.discretisation import EqualWidthDiscretiser
from feature_engine.imputation import MeanMedianImputer,CategoricalImputer

In [87]:
data_save_path = './data/'
root_path = r'C:\\Users\\8prab\\Documents\\NSE_Equity_Futures_iEOD\\'
pattern = '*BANKNIFTY.txt'
data_name = 'banknifty'
#base_df= de.create_dataset(root_path,pattern,data_save_path,data_name)
base_df = pd.read_csv(r'C:\Users\8prab\Google Drive\Work\trading\data\banknifty\banknifty.csv')

In [88]:
base_df.shape

(1170729, 4)

In [4]:
from sklearn.pipeline import Pipeline
pipe = Pipeline([('labelgenerator_1', de.LabelCreator(freq='1min',shift=-15,shift_column='close')),('labelgenerator_2', de.LabelCreator(freq='1min',shift=-30,shift_column='close')),('mean_imputer', MeanMedianImputer()),('cat_imputer', CategoricalImputer())])
base_df = pipe.fit(base_df).transform(base_df)

In [5]:
base_df.head()

,open,high,low,close,label_-15_1min_close,label_-30_1min_close
2010-03-02 09:01:00,8826.90,8856.95,8826.90,8839.80,40to60,40to60
2010-03-02 09:02:00,8840.75,8853.90,8832.80,8852.30,40to60,20to40
2010-03-02 09:03:00,8855.70,8855.70,8844.65,8846.65,40to60,40to60
2010-03-02 09:04:00,8843.85,8850.70,8843.75,8846.70,40to60,20to40
2010-03-02 09:05:00,8846.40,8846.40,8837.75,8841.60,60to80,40to60


In [9]:
obj = EqualWidthDiscretiser(variables=['open'], bins=10, return_object=False, return_boundaries=False)
obj.fit(base_df.fillna(0)).transform(base_df)

,open,high,low,close,label_-15_1min_close,label_-30_1min_close
2010-03-02 09:01:00,1,8856.95,8826.90,8839.80,40to60,40to60
2010-03-02 09:02:00,1,8853.90,8832.80,8852.30,40to60,20to40
2010-03-02 09:03:00,1,8855.70,8844.65,8846.65,40to60,40to60
2010-03-02 09:04:00,1,8850.70,8843.75,8846.70,40to60,20to40
2010-03-02 09:05:00,1,8846.40,8837.75,8841.60,60to80,40to60
...,...,...,...,...,...,...
2022-01-13 15:28:00,9,38469.15,38459.45,38467.20,unknown,unknown
2022-01-13 15:29:00,9,38479.85,38456.65,38456.65,unknown,unknown
2022-01-13 15:30:00,9,38471.55,38455.15,38465.70,unknown,unknown
2022-01-13 15:31:00,9,38466.05,38466.05,38466.05,unknown,unknown


In [8]:
pipe.get_params()

{'memory': None,
 'steps': [('labelgenerator_1', LabelCreator()),
  ('labelgenerator_2', LabelCreator(shift=-30)),
  ('technical', TechnicalIndicator())],
 'verbose': False,
 'labelgenerator_1': LabelCreator(),
 'labelgenerator_2': LabelCreator(shift=-30),
 'technical': TechnicalIndicator(),
 'labelgenerator_1__freq': '1min',
 'labelgenerator_1__shift': -15,
 'labelgenerator_1__shift_column': 'close',
 'labelgenerator_2__freq': '1min',
 'labelgenerator_2__shift': -30,
 'labelgenerator_2__shift_column': 'close'}

In [9]:
pipe.set_params()

Pipeline(steps=[('labelgenerator_1', LabelCreator()),
                ('labelgenerator_2', LabelCreator(shift=-30)),
                ('technical', TechnicalIndicator())])

In [10]:
from feature_engine.discretisation import EqualWidthDiscretiser
from feature_engine.imputation import MeanMedianImputer

In [11]:
base_df.columns

Index(['open', 'high', 'low', 'close', 'label_-15_1min_close',
       'label_-30_1min_close', 'momentum_ADX', 'momentum_ADXR', 'momentum_APO',
       'momentum_BOP',
       ...
       'overlap_double_exp_moving_avg', 'overlap_exp_moving_avg',
       'overlap_hilbert_transform', 'overlap_kaufman_adaptive_moving_avg',
       'overlap_moving_avg', 'overlap_midpoint', 'overlap_midpoint_price',
       'overlap_simple_moving_avg', 'overlap_triangular_moving_average',
       'overlap_weighted_moving_average'],
      dtype='object', length=141)

ValueError: Some of the variables to transform contain inf values. Check and remove those before using this transformer.

In [29]:
n = 2
x = np.concatenate([[np.nan] * (n), base_df['close'].values])

def rolling_window(a, window):
    shape = a.shape[:-1] + (a.shape[-1] - window + 1, window)
    strides = a.strides + (a.strides[-1],)
    return np.lib.stride_tricks.as_strided(a, shape=shape, strides=strides)
arr = rolling_window(x, n + 1)

base_df['new'] = (arr[:, :-1] > arr[:, [-1]]).sum(axis=1)

<ipython-input-29-402c5888b12e>:10: RuntimeWarning: invalid value encountered in greater
  base_df['new'] = (arr[:, :-1] > arr[:, [-1]]).sum(axis=1)


In [31]:
base_df

,open,high,low,close,label_-15_1min_close,label_-30_1min_close,new
2010-03-02 09:01:00,8826.90,8856.95,8826.90,8839.80,40to60,40to60,0
2010-03-02 09:02:00,8840.75,8853.90,8832.80,8852.30,40to60,20to40,0
2010-03-02 09:03:00,8855.70,8855.70,8844.65,8846.65,40to60,40to60,1
2010-03-02 09:04:00,8843.85,8850.70,8843.75,8846.70,40to60,20to40,1
2010-03-02 09:05:00,8846.40,8846.40,8837.75,8841.60,60to80,40to60,2
...,...,...,...,...,...,...,...
2022-01-13 15:28:00,38464.35,38469.15,38459.45,38467.20,unknown,unknown,1
2022-01-13 15:29:00,38469.00,38479.85,38456.65,38456.65,unknown,unknown,2
2022-01-13 15:30:00,38464.05,38471.55,38455.15,38465.70,unknown,unknown,1
2022-01-13 15:31:00,38466.05,38466.05,38466.05,38466.05,unknown,unknown,0


In [34]:
last_ticks = 2
base_df = base_df.replace([np.inf, -np.inf], np.nan).dropna(subset=["close"], how="all")
base_df['new1'] = (base_df['close'].rolling(last_ticks+1, min_periods=1)
                        .apply(lambda x: (x[-1] > x[:-1]).mean())
                        .astype(int))
base_df

ValueError: Cannot convert non-finite values (NA or inf) to integer

In [25]:
import numpy as np
df = pd.DataFrame({"a": np.random.randint(1, high=10, size=50)})
#print(df)
def f(s):
    #d = s.to_frame().groupby(pd.cut(s, np.arange(s.min(), s.max()*1.01, 1), include_lowest=True)).transform("count")
    tmp, bins = pd.cut(
                x=s, bins=3, retbins=True, duplicates="drop"
            )
    bins = list(bins)
    bins[0] = float("-inf")
    bins[len(bins) - 1] = float("inf")
    return pd.cut(x=s, bins=bins, labels=False)

df["a"].rolling(10).apply(f)

TypeError: cannot convert the series to <class 'float'>

In [21]:
tmp, bins = pd.cut(
                x=df['a'], bins=3, retbins=True, duplicates="drop"
            )
bins

array([0.992     , 3.66666667, 6.33333333, 9.        ])

In [22]:
bins = list(bins)
bins[0] = float("-inf")
bins[len(bins) - 1] = float("inf")
bins

[-inf, 3.6666666666666665, 6.333333333333333, inf]

In [24]:
t = pd.cut(x=df['a'], bins=bins, labels=False)
t

0     0
1     2
2     2
3     2
4     0
5     1
6     2
7     2
8     1
9     2
10    0
11    0
12    2
13    2
14    0
15    1
16    1
17    1
18    1
19    0
20    1
21    0
22    0
23    2
24    1
25    1
26    2
27    2
28    1
29    1
30    2
31    0
32    0
33    2
34    2
35    1
36    0
37    1
38    2
39    0
40    1
41    1
42    0
43    0
44    1
45    0
46    1
47    2
48    1
49    0
Name: a, dtype: int64

In [30]:
i = pd.date_range('2018-04-09', periods=100, freq='30min')
ts = pd.DataFrame({'A': list(range(100))}, index=i)
ts

,A
2018-04-09 00:00:00,0
2018-04-09 00:30:00,1
2018-04-09 01:00:00,2
2018-04-09 01:30:00,3
2018-04-09 02:00:00,4
...,...
2018-04-10 23:30:00,95
2018-04-11 00:00:00,96
2018-04-11 00:30:00,97
2018-04-11 01:00:00,98


In [32]:
ts.between_time('09:00', '10:30')['A']

2018-04-09 09:00:00    18
2018-04-09 09:30:00    19
2018-04-09 10:00:00    20
2018-04-09 10:30:00    21
2018-04-10 09:00:00    66
2018-04-10 09:30:00    67
2018-04-10 10:00:00    68
2018-04-10 10:30:00    69
Name: A, dtype: int64

In [63]:
from datetime import datetime


In [83]:
s1 = ts.between_time('09:00', '10:30').groupby(pd.Grouper(freq='d')).max() - ts.between_time('09:00', '10:30').groupby(pd.Grouper(freq='d')).min()
s1 = ts['A'].between_time('09:00', '10:30').groupby(pd.Grouper(freq='d')).max() 
#s1.index  =str(s1.index) + '10:30:00'	
s1.index = pd.to_datetime(s1.index) 
s1 = s1.sort_index()
s1.index = s1.index + pd.DateOffset(minutes=60*10.5)
s1

2018-04-09 10:30:00    21
2018-04-10 10:30:00    69
Name: A, dtype: int64

In [71]:
s1.index

DatetimeIndex(['2018-04-09', '2018-04-10'], dtype='datetime64[ns]', freq='D')

In [42]:
s1.dtypes

A    int64
dtype: object

In [84]:
merge=pd.merge(ts,s1, how='outer', left_index=True, right_index=True)
merge = merge.fillna(method='ffill')
#merge

In [85]:
merge.between_time('09:00', '12:30')

,A_x,A_y
2018-04-09 09:00:00,18,NaN
2018-04-09 09:30:00,19,NaN
2018-04-09 10:00:00,20,NaN
2018-04-09 10:30:00,21,21.0
2018-04-09 11:00:00,22,21.0
2018-04-09 11:30:00,23,21.0
2018-04-09 12:00:00,24,21.0
2018-04-09 12:30:00,25,21.0
2018-04-10 09:00:00,66,21.0
2018-04-10 09:30:00,67,21.0


In [89]:
base_df.head()

,open,high,low,close
20180402 09:08,24235.7,24235.7,24235.7,24235.7
20180402 09:16,24268.0,24300.6,24243.7,24253.1
20180402 09:17,24257.4,24286.1,24219.3,24221.4
20180402 09:18,24222.6,24224.4,24204.0,24220.7
20180402 09:19,24218.7,24247.1,24218.7,24229.4


In [92]:
base_df.index = pd.to_datetime(base_df.index)
base_df = base_df.sort_index()
base_df = base_df[~base_df.index.duplicated(keep='first')]  
s1 = base_df['high'].between_time('09:00', '10:30').groupby(pd.Grouper(freq='d')).max() - base_df['low'].between_time('09:00', '10:30').groupby(pd.Grouper(freq='d')).min()
#s1 = ts['A'].between_time('09:00', '10:30').groupby(pd.Grouper(freq='d')).max() 
#s1.index  =str(s1.index) + '10:30:00'	
s1.index = pd.to_datetime(s1.index) 
s1 = s1.sort_index()
c = [int(i) for i in x.split(':')]
s1.index = s1.index + pd.DateOffset(minutes=c[0]*60 + c[1])
pd.DataFrame(s1)

2010-03-02 10:30:00     82.70
2010-03-03 10:30:00    101.85
2010-03-04 10:30:00     53.90
2010-03-05 10:30:00     82.45
2010-03-06 10:30:00       NaN
                        ...  
2022-01-09 10:30:00       NaN
2022-01-10 10:30:00    332.25
2022-01-11 10:30:00    453.00
2022-01-12 10:30:00    246.85
2022-01-13 10:30:00    301.50
Freq: D, Length: 4336, dtype: float64

In [98]:
s1 = pd.DataFrame(s1,columns=['range_1030'])

In [100]:
pd.options.display.max_rows = 100

In [106]:
merge=pd.merge(base_df,s1, how='outer', left_index=True, right_index=True)
merge['range_1030'] = merge['range_1030'].fillna(method='ffill')
merge

,open,high,low,close,range_1030
2010-03-02 09:01:00,8826.90,8856.95,8826.90,8839.80,NaN
2010-03-02 09:02:00,8840.75,8853.90,8832.80,8852.30,NaN
2010-03-02 09:03:00,8855.70,8855.70,8844.65,8846.65,NaN
2010-03-02 09:04:00,8843.85,8850.70,8843.75,8846.70,NaN
2010-03-02 09:05:00,8846.40,8846.40,8837.75,8841.60,NaN
...,...,...,...,...,...
2022-01-13 15:28:00,38464.35,38469.15,38459.45,38467.20,301.5
2022-01-13 15:29:00,38469.00,38479.85,38456.65,38456.65,301.5
2022-01-13 15:30:00,38464.05,38471.55,38455.15,38465.70,301.5
2022-01-13 15:31:00,38466.05,38466.05,38466.05,38466.05,301.5


In [108]:
merge.between_time('10:29', '10:31')

,open,high,low,close,range_1030
2010-03-02 10:29:00,8864.05,8868.35,8863.60,8867.20,NaN
2010-03-02 10:30:00,8865.05,8868.80,8865.05,8868.25,82.70
2010-03-02 10:31:00,8867.70,8870.95,8867.70,8869.30,82.70
2010-03-03 10:29:00,9020.85,9020.85,9017.75,9018.55,82.70
2010-03-03 10:30:00,9020.10,9021.00,9019.05,9020.50,101.85
...,...,...,...,...,...
2022-01-12 10:30:00,38835.50,38841.40,38828.80,38840.60,246.85
2022-01-12 10:31:00,38838.20,38840.65,38818.35,38828.90,246.85
2022-01-13 10:29:00,38475.45,38478.15,38459.95,38464.00,246.85
2022-01-13 10:30:00,38461.60,38465.05,38455.30,38458.15,301.50


In [109]:
import datetime as DT
t1 = DT.datetime.strptime('00:05:36.0100000', '%H:%M:%S.%f0')
t2 = DT.datetime(1900,1,1)

print((t1-t2).total_seconds() / 60.0)

5.6001666666666665


In [122]:
x = '10:30'
c = [int(i) for i in x.split(':')]

In [124]:
c[0]*60 + c[1]

630

In [121]:
x.split(':')

['10', '30']